In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import os
import glob         

In [17]:
# credible; makes dictionary of datasets sorted by user (credible), and dataframe of all credible tweets with username as additional column (credible_df) 
df_path = glob.glob('C:/Users/Maggie/Documents/BDC/bdc2021/Datasets/Credible/*.csv')
credible = {}
df_ls = []

for path in df_ls:
    df = pd.read_csv(path)
    df = df[['name','tweet','mentions','urls','photos','replies_count','retweets_count','likes_count','hashtags','retweet']]

    # general keywords used for dataset 4: ncov19, ncov2019, covid, rona, stayathome, socialdistancing
    covid = df.loc[df['tweet'].str.contains('covid|coronavirus|rona|wearamask|socialdistancing|stayathome|ncov|symptomatic|covax', case=False)]

    df_ls.append(covid)
    credible[path.split('\\')[-1][:-4]] = covid.drop(['name'], axis = 1)

credible_df = pd.concat(df_ls, ignore_index = True)

In [ ]:
# News_Outlets
df_path = glob.glob('C:/Users/Maggie/Documents/BDC/bdc2021/Datasets/Credible/*.csv')
news = {}
df_ls = []

for path in df_ls:
    df = pd.read_csv(path)
    df = df[['name','tweet','mentions','urls','photos','replies_count','retweets_count','likes_count','hashtags','retweet']]

    # general keywords used for dataset 4: ncov19, ncov2019, covid, rona, stayathome, socialdistancing
    covid = df.loc[df['tweet'].str.contains('covid|coronavirus|rona|wearamask|socialdistancing|stayathome|ncov|symptomatic|covax', case=False)]
    
    df_ls.append(covid)
    news[path.split('\\')[-1][:-4]] = covid.drop(['name'], axis = 1)

news_df = pd.concat(df_ls, ignore_index = True)

    

In [ ]:
# Individual
df_path = glob.glob('C:/Users/Maggie/Documents/BDC/bdc2021/Datasets/Political/*.csv')
ind = {}
df_ls = []

for path in df_ls:
    df = pd.read_csv(path)
    df = df[['name','tweet','mentions','urls','photos','replies_count','retweets_count','likes_count','hashtags','retweet']]

    # general keywords used for dataset 4: ncov19, ncov2019, covid, rona, stayathome, socialdistancing
    covid = df.loc[df['tweet'].str.contains('covid|coronavirus|rona|wearamask|socialdistancing|stayathome|ncov|symptomatic|covax', case=False)]
    
    df_ls.append(covid)
    news[path.split('\\')[-1][:-4]] = covid.drop(['name'], axis = 1)

ind_df = pd.concat(df_ls, ignore_index = True)
